<a href="https://colab.research.google.com/github/joaopdss/segmentation-system/blob/main/semantic_segmentation_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!apt-get install p7zip-full

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


In [3]:
!7za x /content/drive/MyDrive/Datasets/mapillary.zip


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/Datasets/                                          1 file, 4675505893 bytes (4459 MiB)

Extracting archive: /content/drive/MyDrive/Datasets/mapillary.zip
  0% 256 Open               0% 512 Open               1% 768 Open               1% 1024 Open                2% 1536 Open                3% 2048 Open                3% 2304 Open                6% 4352 Open                7% 4608 Open                

In [4]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Dropout, MaxPooling2D, Conv2DTranspose, concatenate, Input, Cropping2D
import matplotlib.pyplot as plt
from PIL import Image
import json

In [13]:
labels = ["person", "traffic", "vehicle", "truck", "trailer", "motorcycle", "car", "bus", "bicycle", "trash can",
 "bench", "banner", "water", "water", "vegetation", "terrain", "sky", "sand", "mountain", "lane", "tunnel",
 "building", "bridge", "sidewalk", "road", "crosswalk", "wall"]

In [34]:
# get 'readable' and 'color' based on labels i want]
f = open("/content/mapillary/config_v2.0.json")
config = json.load(f)
keys = config['labels']
matches = []

for label in labels:
  matches += [{item['readable']: item['color']} for item in keys if label in item['name'].lower()]

In [35]:
matches[:5]

[{'Person': [220, 20, 60]},
 {'Person Group': [220, 20, 60]},
 {'Traffic Island': [128, 196, 128]},
 {'Traffic Sign Frame': [128, 128, 128]},
 {'Traffic Cone': [210, 60, 60]}]

In [ ]:
with open('/content/mapillary/config_v2.0.json', 'r') as f:
    config = json.load(f)

labels = config['labels']
print(f"Amount of labels {len(labels)}")
color_map = {}

for label in labels:
  color_map[label['readable']] = label['color']

label_list = sorted(color_map.keys())
train_ids = {k:v for v,k in enumerate(label_list)}

Amount of labels 124


In [ ]:
train_ids

In [ ]:
from tensorflow.keras import backend as K
print(K.image_data_format()) # print current format
K.set_image_data_format('channels_last')

channels_last


In [ ]:
import os
print(len(os.listdir("/content/mapillary/training/images")))
print(len(os.listdir("/content/mapillary/validation/images")))
print(len(os.listdir("/content/mapillary/training/v2.0/labels")))
print(len(os.listdir("/content/mapillary/validation/v2.0/labels")))

17999
2000
17999
2000


In [ ]:
import gc

def read_process(train_image_path, train_mask_path):
  train_set, val_set = None, None

  for dir in ["training", "validation"]:
    path_img = train_image_path.format(dir)
    path_mask = train_image_path.format(dir)

    count = 0
    images = []
    masks = []

    for image in os.listdir(path_img):
      if count <= 100:
        img = tf.io.read_file(f"{path_img}/{image}")
        img = tf.io.decode_png(img)
        # img = tf.cast(img, tf.float32) / 255.0
        with tf.device("CPU"):
          img = tf.image.convert_image_dtype(img, tf.float16)
        # img = tf.cast(img, tf.float32)
        # print(img)
        images.append(img)
        count += 1
        break
      else:
        count = 0
        break

    for mask in os.listdir(path_mask):
      if count <= 100:
        mask = tf.io.read_file(f"{path_mask}/{image}")
        mask = tf.io.decode_png(mask)
        with tf.device("CPU"):
          mask = tf.image.rgb_to_grayscale(mask)
        # mask = tf.cast(mask, tf.float32)
        masks.append(mask)
        count += 1
        break
      else:
        break

    if dir == "training":
        train_set = tf.data.Dataset.from_tensor_slices((images, masks)).prefetch(tf.data.AUTOTUNE)
    else:
      val_set = tf.data.Dataset.from_tensor_slices((images, masks)).prefetch(tf.data.AUTOTUNE)

  return train_set, val_set

In [ ]:
train_set, val_set = read_process("/content/mapillary/{0}/images", "/content/mapillary/{0}/v2.0/labels")

In [ ]:
for x, y in train_set:
  print(y[0][2])

tf.Tensor(
[[ 74]
 [ 77]
 [ 88]
 [ 80]
 [ 85]
 [ 85]
 [ 88]
 [ 93]
 [ 84]
 [ 82]
 [ 81]
 [ 85]
 [ 86]
 [ 86]
 [ 89]
 [ 89]
 [ 85]
 [ 89]
 [ 93]
 [ 85]
 [ 88]
 [102]
 [ 35]
 [ 38]
 [ 74]
 [ 80]
 [ 84]
 [ 82]
 [ 70]
 [ 85]
 [ 87]
 [ 87]
 [ 87]
 [ 89]
 [ 91]
 [ 86]
 [ 88]
 [ 85]
 [ 85]
 [ 91]
 [ 90]
 [ 85]
 [ 86]
 [ 85]
 [ 82]
 [ 75]
 [ 82]
 [ 76]
 [ 80]
 [ 72]
 [ 68]
 [ 73]
 [ 73]
 [ 73]
 [ 77]
 [ 79]
 [ 74]
 [ 66]
 [ 69]
 [112]
 [179]
 [189]
 [189]
 [185]
 [180]
 [179]
 [180]
 [179]
 [181]
 [177]
 [183]
 [177]
 [177]
 [178]
 [178]
 [179]
 [180]
 [179]
 [180]
 [178]
 [180]
 [181]
 [181]
 [182]
 [185]
 [189]
 [189]
 [186]
 [186]
 [182]
 [183]
 [185]
 [185]
 [184]
 [183]
 [182]
 [185]
 [185]
 [184]
 [183]
 [181]
 [181]
 [182]
 [183]
 [183]
 [183]
 [183]
 [181]
 [181]
 [182]
 [183]
 [182]
 [183]
 [184]
 [185]
 [184]
 [185]
 [186]
 [188]
 [188]
 [186]
 [186]
 [186]
 [184]
 [185]
 [186]
 [187]
 [187]
 [188]
 [188]
 [189]
 [190]
 [191]
 [192]
 [193]
 [194]
 [195]
 [194]
 [194]
 [195]
 [195]
 [

In [ ]:
train_set = train_set.batch(32)
val_set = val_set.batch(32)

In [ ]:
for i in train_set.batch(32).take(1):
  fig, axes = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(5,5))

  axes[0].imshow(i[0][0] * 255)
  axes[1].imshow(i[0][1] * 255)
  break

## Implementing U-Net architecture

In [ ]:
def encoder_block(inputs, n_filters=64, dropout_prob=0.5, max_pooling=True):
  conv = Conv2D(n_filters,
                3,
                activation='relu',
                padding='same',
                kernel_initializer='HeNormal')(inputs)
  conv = Conv2D(n_filters,
                3,
                activation='relu',
                padding='same',
                kernel_initializer='HeNormal')(conv)
  conv = BatchNormalization()(conv, training=False)
  conv = Dropout(dropout_prob)(conv)

  if max_pooling:
    next_layer = MaxPooling2D(pool_size=(2,2))(conv)
  else:
    next_layer = conv

  skip_connection = conv
  return next_layer, skip_connection


def decoder_block(prev_layer_input, skip_layer_input, n_filters=64):
  up = Conv2DTranspose(n_filters,
                       (3,3),
                       strides=(2,2),
                       padding='same')(prev_layer_input)

  merge = concatenate([up, skip_layer_input], axis=-1)

  conv = Conv2D(n_filters,
                3,
                activation='relu',
                padding='same',
                kernel_initializer='HeNormal')(merge)
  conv = Conv2D(n_filters,
                3,
                activation='relu',
                padding='same',
                kernel_initializer='HeNormal')(conv)
  return conv

In [ ]:
def unet_compiled(input_size=(256, 512, 3), n_filters=64, n_classes=3):
  inputs = Input(input_size)

  # Encoder
  cblock1 = encoder_block(inputs, n_filters,dropout_prob=0, max_pooling=True)
  cblock2 = encoder_block(cblock1[0],n_filters*2,dropout_prob=0, max_pooling=True)
  cblock3 = encoder_block(cblock2[0], n_filters*4,dropout_prob=0, max_pooling=True)
  cblock4 = encoder_block(cblock3[0], n_filters*8,dropout_prob=0.2, max_pooling=True)
  cblock5 = encoder_block(cblock4[0], n_filters*16, dropout_prob=0.4, max_pooling=False)

  # Decoder
  ublock6 = decoder_block(cblock5[0], cblock4[1], n_filters * 8)
  ublock7 = decoder_block(ublock6, cblock3[1], n_filters * 4)
  ublock8 = decoder_block(ublock7, cblock2[1], n_filters * 2)
  ublock9 = decoder_block(ublock8, cblock1[1], n_filters)

  conv9 = Conv2D(n_filters,
                 3,
                 activation='relu',
                 padding='same',
                 kernel_initializer='he_normal')(ublock9)

  conv10 = Conv2D(n_classes, 1, padding='same', activation="softmax")(conv9)

  # Define the model
  model = tf.keras.Model(inputs=inputs, outputs=conv10)

  return model

In [ ]:
unet = unet_compiled(input_size=(256,512,3), n_filters=64, n_classes=124)

In [ ]:
unet.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 256, 512, 3)]        0         []                            
                                                                                                  
 conv2d_40 (Conv2D)          (None, 256, 512, 64)         1792      ['input_3[0][0]']             
                                                                                                  
 conv2d_41 (Conv2D)          (None, 256, 512, 64)         36928     ['conv2d_40[0][0]']           
                                                                                                  
 batch_normalization_10 (Ba  (None, 256, 512, 64)         256       ['conv2d_41[0][0]']           
 tchNormalization)                                                                          

In [ ]:
unet.compile(optimizer=tf.keras.optimizers.Adam(),
             loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
results = unet.fit(train_set, batch_size=32, epochs=10, validation_data=val_set)

Epoch 1/10
1/1 [==============================] - 11s 11s/step - loss: nan - accuracy: 0.0059 - val_loss: nan - val_accuracy: 1.5259e-04
Epoch 2/10
1/1 [==============================] - 0s 304ms/step - loss: nan - accuracy: 0.0364 - val_loss: nan - val_accuracy: 1.5259e-04
Epoch 3/10
1/1 [==============================] - 0s 234ms/step - loss: nan - accuracy: 0.0364 - val_loss: nan - val_accuracy: 1.5259e-04
Epoch 4/10
1/1 [==============================] - 0s 233ms/step - loss: nan - accuracy: 0.0364 - val_loss: nan - val_accuracy: 1.5259e-04
Epoch 5/10
1/1 [==============================] - 0s 241ms/step - loss: nan - accuracy: 0.0364 - val_loss: nan - val_accuracy: 1.5259e-04
Epoch 6/10
1/1 [==============================] - 0s 232ms/step - loss: nan - accuracy: 0.0364 - val_loss: nan - val_accuracy: 1.5259e-04
Epoch 7/10
1/1 [==============================] - 0s 232ms/step - loss: nan - accuracy: 0.0364 - val_loss: nan - val_accuracy: 1.5259e-04
Epoch 8/10
1/1 [===================

In [ ]:
unet.save("unet.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
def unet_test():
    inputs = Input(shape=(572, 572, 3))
    # first part of the U - contracting part
    c0 = Conv2D(64, activation='relu', kernel_size=3)(inputs)
    c1 = Conv2D(64, activation='relu', kernel_size=3)(c0)  # This layer for concatenating in the expansive part
    c2 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(c1)
    print(c2)

    c3 = Conv2D(128, activation='relu', kernel_size=3)(c2)
    c4 = Conv2D(128, activation='relu', kernel_size=3)(c3)  # This layer for concatenating in the expansive part
    c5 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(c4)
    print(c5)

    c6 = Conv2D(256, activation='relu', kernel_size=3)(c5)
    c7 = Conv2D(256, activation='relu', kernel_size=3)(c6)  # This layer for concatenating in the expansive part
    c8 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(c7)
    print(c8)

    c9 = Conv2D(512, activation='relu', kernel_size=3)(c8)
    c10 = Conv2D(512, activation='relu', kernel_size=3)(c9)  # This layer for concatenating in the expansive part
    c11 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(c10)
    print(c11)

    c12 = Conv2D(1024, activation='relu', kernel_size=3)(c11)
    c13 = Conv2D(1024, activation='relu', kernel_size=3, padding='valid')(c12)

    # We will now start the second part of the U - expansive part
    t01 = Conv2DTranspose(512, kernel_size=2, strides=(2, 2), activation='relu')(c13)
    crop01 = Cropping2D(cropping=(4, 4))(c10)
    print(f"prev {c13}")
    print(f"skip {c10}")
    print(t01)
    print(crop01)
    concat01 = concatenate([t01, crop01], axis=-1)

    c14 = Conv2D(512, activation='relu', kernel_size=3)(concat01)
    c15 = Conv2D(512, activation='relu', kernel_size=3)(c14)

    t02 = Conv2DTranspose(256, kernel_size=2, strides=(2, 2), activation='relu')(c15)
    crop02 = Cropping2D(cropping=(16, 16))(c7)

    concat02 = concatenate([t02, crop02], axis=-1)

    c16 = Conv2D(256, activation='relu', kernel_size=3)(concat02)
    c17 = Conv2D(256, activation='relu', kernel_size=3)(c16)

    t03 = Conv2DTranspose(128, kernel_size=2, strides=(2, 2), activation='relu')(c17)
    crop03 = Cropping2D(cropping=(40, 40))(c4)

    concat03 = concatenate([t03, crop03], axis=-1)

    c18 = Conv2D(128, activation='relu', kernel_size=3)(concat03)
    c19 = Conv2D(128, activation='relu', kernel_size=3)(c18)

    t04 = Conv2DTranspose(64, kernel_size=2, strides=(2, 2), activation='relu')(c19)
    crop04 = Cropping2D(cropping=(88, 88))(c1)

    concat04 = concatenate([t04, crop04], axis=-1)

    c20 = Conv2D(64, activation='relu', kernel_size=3)(concat04)
    c21 = Conv2D(64, activation='relu', kernel_size=3)(c20)

    # This is based on our dataset. The output channels are 3, think of it as each pixel will be classified
    # into three classes, but I have written 4 here, as I do padding with 0, so we end up have four classes.
    outputs = Conv2D(n_classes, kernel_size=1)(c21)

    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="u-netmodel")
    return model